# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=48

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==48]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm48', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm48/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-08-12 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31676/1820351328.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31676/1820351328.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31676/1820351328.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
                [,1]      [,2]         [,3]        [,4]
sigmas -0.9383058845 1.6297002 -5.686294694 23.50612143
       -0.9256390615 1.6024561 -5.578052120 23.34363197
       -0.9131306820 1.5756568 -5.469812543 23.17653049
       -0.9007768311 1.5493095 -5.361644703 23.00502279
       -0.8885737374 1.5234205 -5.253614422 22.82930872
       -0.8765177656 1.4979956 -5.145784685 22.64958227
       -0.8646054105 1.4730396 -5.038215729 22.46603164
       -0.8528332907 1.4485570 -4.930965123 22.27883936
       -0.8411981429 1.4245513 -4.824087855 22.08818241
       -0.8296968162 1.4010259 -4.717636414 21.89423228
       -0.8183262673 1.3779831 -4.611660868 21.69715515
       -0.8070835555 1.3554251 -4.506208950 21.49711193
       -0.7959658384 1.3333532 -4.401326131 21.29425844
       -0.7849703670 1.3117685 -4.297055701 21.08874545
       -0.7740944824 1.2906715 -4.193438843 20.88071890
       -0.76333

...

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]         [,4]
sigmas -0.743448020 1.0868209 -1.461493822  3.251416321
       -0.728762566 1.0764700 -1.437263071  3.221519712
       -0.714289657 1.0662967 -1.413281592  3.191812956
       -0.700023228 1.0562991 -1.389547799  3.162297212
       -0.685957471 1.0464752 -1.366060068  3.132973523
       -0.672086818 1.0368232 -1.342816744  3.103842823
       -0.658405931 1.0273410 -1.319816140  3.074905931
       -0.644909687 1.0180269 -1.297056536  3.046163553
       -0.631593170 1.0088788 -1.274536188  3.017616285
       -0.618451654 0.9998950 -1.252253322  2.989264608
       -0.605480599 0.9910735 -1.230206142  2.961108896
       -0.592675641 0.9824125 -1.208392826  2.933149412
       -0.580032579 0.9739102 -1.186811534  2.905386311
       -0.567547370 0.9655647 -1.165460406  2.877819639
       -0.555216121 0.9573743 -1.144337564  2.850449341
       -0.543035082 0.9493371 -1.123441118  2.823275255
       -0.5310

        0.397683819 0.7149505  0.232945651  0.391585032
        0.402398677 0.7165986  0.235645485  0.376071908
        0.407091411 0.7182660  0.238237110  0.360539162
        0.411762225 0.7199519  0.240720168  0.344985163
        0.416411324 0.7216556  0.243094283  0.329408272
        0.421038910 0.7233763  0.245359054  0.313806842
        0.425645179 0.7251133  0.247514061  0.298179216
        0.430230327 0.7268658  0.249558860  0.282523730
        0.434794549 0.7286331  0.251492985  0.266838707
        0.439338032 0.7304143  0.253315948  0.251122461
        0.443860966 0.7322086  0.255027237  0.235373296
        0.448363534 0.7340154  0.256626318  0.219589503
        0.452845921 0.7358337  0.258112634  0.203769363
        0.457308305 0.7376628  0.259485603  0.187911143
        0.461750865 0.7395019  0.260744620  0.172013100
        0.466173776 0.7413501  0.261889056  0.156073475
        0.470577211 0.7432067  0.262918259  0.140090496
        0.474961340 0.7450708  0.263831548  0.12

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31676/1820351328.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31676/1820351328.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31676/1820351328.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
                [,1]      [,2]         [,3]         [,4]
sigmas -0.8708274277 1.1442835 -1.726083298  3.272549578
       -0.8514278498 1.1302370 -1.699318967  3.250448897
       -0.8323974647 1.1164085 -1.672730178  3.228323323
       -0.8137224823 1.1027968 -1.646317664  3.206174883
       -0.7953898708 1.0894005 -1.620082112  3.184005526
       -0.7773873025 1.0762182 -1.594024161  3.161817121
       -0.7597031036 1.0632485 -1.568144411  3.139611459
       -0.7423262091 1.0504901 -1.542443417  3.117390255
       -0.7252461208 1.0379415 -1.516921699  3.095155149
       -0.7084528696 1.0256014 -1.491579734  3.072907710
       -0.6919369802 1.0134682 -1.466417968  3.050649436
       -0.6756894394 1.0015406 -1.441436808  3.028381753
       -0.6597016663 0.9898171 -1.416636633  3.006106025
       -0.6439654849 0.9782963 -1.392017788  2.983823546
       -0.6284730995 0.9669767 -1.367580589  2.961535551
       -0.6132170710 0.9558569 -1.343325324  2.9392432

        0.4593287860 0.6456367  0.268683428  0.194188389
        0.4644954444 0.6478356  0.270169931  0.173353757
        0.4696355455 0.6500460  0.271486994  0.152525159
        0.4747493610 0.6522667  0.272634660  0.131702281
        0.4798371584 0.6544961  0.273612968  0.110884805
        0.4848992012 0.6567331  0.274421950  0.090072411
        0.4899357486 0.6589761  0.275061634  0.069264774
        0.4949470563 0.6612238  0.275532039  0.048461566
        0.4999333760 0.6634749  0.275833181  0.027662455
        0.5048949556 0.6657279  0.275965069  0.006867109
        0.5098320394 0.6679815  0.275927708 -0.013924810
        0.5147448681 0.6702344  0.275721094 -0.034713642
        0.5196336789 0.6724851  0.275345219 -0.055499727
        0.5244987054 0.6747322  0.274800070 -0.076283412
        0.5293401780 0.6769745  0.274085625 -0.097065041
        0.5341583237 0.6792104  0.273201859 -0.117844964
        0.5389533660 0.6814387  0.272148739 -0.138623530
        0.5437255257 0.6836580 

In [2]:
print(error_i)

[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 14, 15, 16, 17, 19, 25, 49]
